In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import pandas as pd
from shapely.ops import unary_union
from shapely.geometry import Point
from scipy.spatial import distance_matrix

In [2]:
# Load the shapefile
shapefile_path = 'C:/Users/laura/Desktop/SocialHousing/data/raw/lor_planungsraeume_2021.shp/lor_planungsraeume_2021.shp'
gdf = gpd.read_file(shapefile_path)

In [10]:

# Calculate centroids
gdf['centroid'] = gdf.geometry.centroid

# Extract centroid coordinates and PLR_IDs
centroid_coords = gdf['centroid'].apply(lambda x: (x.x, x.y)).tolist()
plr_ids = gdf['PLR_ID'].tolist()

# Calculate distance matrix
dist_matrix = distance_matrix(centroid_coords, centroid_coords)

# Convert to DataFrame for easier analysis
dist_df = pd.DataFrame(dist_matrix, index=plr_ids, columns=plr_ids)

# Rename the columns for each unit based on the target PLR_ID
def create_distance_columns(dist_df):
    # Create a copy of the dataframe to avoid modifying the original
    renamed_dist_df = dist_df.copy()
    for plr_id in dist_df.index:
        new_column_names = {col: f'dist_{col}' for col in dist_df.columns}
        renamed_dist_df.rename(columns=new_column_names, inplace=True)
        break  # We only need to do this once
    return renamed_dist_df

# Apply the renaming function to the distance dataframe
renamed_dist_df = create_distance_columns(dist_df)

# Merge the distance information back to the original GeoDataFrame
for target_plr_id in plr_ids:
    column_name = f'dist_{target_plr_id}'
    gdf[column_name] = gdf['PLR_ID'].apply(lambda x: dist_df.loc[x, target_plr_id])

C:\Users\laura\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
C:\Users\laura\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
C:\Users\laura\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many tim

In [12]:
print(gdf.head())


     PLR_ID                  PLR_NAME BEZ       STAND   GROESSE_M2  \
0  11501341            Karlshorst Süd  11  01.01.2021  2294021.723   
1  07601340  Tirschenreuther Ring Ost  07  01.01.2021   413969.326   
2  02500831               Wismarplatz  02  01.01.2021   248988.291   
3  12601134        Märkisches Zentrum  12  01.01.2021  1127855.841   
4  07601547        Horstwalder Straße  07  01.01.2021   941433.954   

                                            geometry  \
0  POLYGON ((400826.929 5814871.126, 400825.707 5...   
1  POLYGON ((387795.701 5807587.153, 387789.731 5...   
2  POLYGON ((396010.552 5819339.570, 396021.567 5...   
3  POLYGON ((387367.428 5828619.763, 387315.862 5...   
4  POLYGON ((391807.174 5805331.900, 391808.186 5...   

                         centroid  dist_11501341  dist_07601340  \
0  POINT (399496.744 5815232.245)       0.000000   13470.107132   
1  POINT (388114.010 5808029.665)   13470.107132       0.000000   
2  POINT (395908.173 5819106.298)    5280

In [15]:
# If you want to save the distance matrix to a CSV file
renamed_dist_df.to_csv('C:/Users/laura/Desktop/SocialHousing/data/temp/distance_matrix.csv')